# Chapter 6: Triangle Rasterization (3D)

In this chapter, we explore how to rasterize 3D triangles with proper depth handling and perspective-correct attribute interpolation. We'll build upon the 2D rasterization techniques from Chapter 2 and integrate them with the 3D viewing pipeline from Chapter 4.

## 6.1 Theory: Perspective-Correct Interpolation

### Linear Interpolation in Screen Space

When we project a 3D triangle onto the screen, we need to interpolate attributes (colors, texture coordinates, normals, etc.) across the triangle. However, simple linear interpolation in screen space is **not correct** for perspective projection.

Given barycentric coordinates $(w_0, w_1, w_2)$ computed in screen space, a naive interpolation of an attribute $a$ would be:

$$a_{\text{wrong}} = w_0 a_0 + w_1 a_1 + w_2 a_2$$

This is incorrect because perspective projection causes non-linear distortion.

### Perspective-Correct Interpolation

The correct approach is to interpolate in **view space** (or world space), then project. However, this is expensive. The practical solution is to use **hyperbolic interpolation**:

1. Divide each attribute by its corresponding depth (z-value):
   $$\frac{a_i}{z_i}$$

2. Linearly interpolate $\frac{a}{z}$ and $\frac{1}{z}$ in screen space:
   $$\frac{a}{z} = w_0 \frac{a_0}{z_0} + w_1 \frac{a_1}{z_1} + w_2 \frac{a_2}{z_2}$$
   $$\frac{1}{z} = w_0 \frac{1}{z_0} + w_1 \frac{1}{z_1} + w_2 \frac{1}{z_2}$$

3. Recover the perspective-correct attribute:
   $$a = \frac{a/z}{1/z}$$

### Mathematical Derivation

Consider a point $\mathbf{p}(t) = (1-t)\mathbf{p}_0 + t\mathbf{p}_1$ linearly interpolated in 3D space. After perspective projection:

$$x_{\text{screen}} = \frac{f \cdot x}{z}$$

If we interpolate $x_{\text{screen}}$ linearly:

$$x_{\text{screen}}(t) = (1-t)\frac{f x_0}{z_0} + t\frac{f x_1}{z_1}$$

This does **not** equal the screen-space projection of the 3D interpolated point:

$$\frac{f((1-t)x_0 + tx_1)}{(1-t)z_0 + tz_1} \neq (1-t)\frac{f x_0}{z_0} + t\frac{f x_1}{z_1}$$

The solution is to interpolate $\frac{x}{z}$ and $\frac{1}{z}$ separately, then divide.

## 6.2 Theory: Depth Interpolation and Z-Fighting

### Depth Value Representation

After projection, the depth value $z$ can be stored in different ways:

1. **View-space depth**: The actual distance from the camera plane
2. **Normalized Device Coordinate (NDC) depth**: Mapped to $[0, 1]$ or $[-1, 1]$
3. **Window-space depth**: Integer values for discrete z-buffer

The perspective projection matrix transforms $z$ non-linearly:

$$z_{\text{NDC}} = \frac{Az + B}{z}$$

where $A$ and $B$ depend on near ($n$) and far ($f$) planes:

$$A = \frac{f + n}{f - n}, \quad B = \frac{-2fn}{f - n}$$

### Depth Precision and Z-Fighting

The non-linear mapping causes **z-fighting**: precision is much higher near the camera than far away. If the ratio $f/n$ is too large, distant objects have insufficient depth precision.

To minimize z-fighting:
- Keep $f/n$ ratio as small as practical (ideally $< 1000$)
- Use floating-point z-buffers instead of integer
- Place near plane as far as possible from the camera

### Barycentric Depth Interpolation

For each fragment with barycentric coordinates $(w_0, w_1, w_2)$:

$$\frac{1}{z} = w_0 \frac{1}{z_0} + w_1 \frac{1}{z_1} + w_2 \frac{1}{z_2}$$

$$z = \frac{1}{1/z}$$

This gives the correct perspective-interpolated depth.

## 6.3 Theory: Vertex Attributes and Shading

### Vertex Attributes

Each vertex in a 3D triangle can have multiple attributes:

- **Position**: $\mathbf{v} = (x, y, z, w)$ in homogeneous coordinates
- **Color**: $(r, g, b)$ or $(r, g, b, a)$
- **Normal**: $\mathbf{n} = (n_x, n_y, n_z)$ for lighting
- **Texture coordinates**: $(u, v)$ for texture mapping
- **Tangent**: $\mathbf{t}$ for normal mapping

All attributes except position must be perspective-corrected during rasterization.

### Flat Shading vs. Smooth Shading

**Flat Shading**: Use a single attribute value (e.g., normal) for the entire triangle.

$$\mathbf{n}_{\text{triangle}} = \frac{(\mathbf{v}_1 - \mathbf{v}_0) \times (\mathbf{v}_2 - \mathbf{v}_0)}{\|(\mathbf{v}_1 - \mathbf{v}_0) \times (\mathbf{v}_2 - \mathbf{v}_0)\|}$$

**Smooth Shading (Gouraud/Phong)**: 
- **Gouraud**: Interpolate colors computed at vertices
- **Phong**: Interpolate normals, compute lighting per fragment

For smooth shading with attribute $\mathbf{a}$:

$$\mathbf{a} = \frac{w_0 \frac{\mathbf{a}_0}{z_0} + w_1 \frac{\mathbf{a}_1}{z_1} + w_2 \frac{\mathbf{a}_2}{z_2}}{w_0 \frac{1}{z_0} + w_1 \frac{1}{z_1} + w_2 \frac{1}{z_2}}$$

After interpolation, normals should be renormalized:

$$\mathbf{n}_{\text{final}} = \frac{\mathbf{n}}{\|\mathbf{n}\|}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, Optional, List

In [ ]:
# Import classes from previous chapters
class Vec3:
    """3D Vector class"""
    def __init__(self, x=0.0, y=0.0, z=0.0):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)
    
    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, scalar):
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def __truediv__(self, scalar):
        return Vec3(self.x / scalar, self.y / scalar, self.z / scalar)
    
    def dot(self, other):
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def cross(self, other):
        return Vec3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )
    
    def length(self):
        return np.sqrt(self.x**2 + self.y**2 + self.z**2)
    
    def normalize(self):
        l = self.length()
        if l > 0:
            return self / l
        return Vec3(0, 0, 0)
    
    def to_array(self):
        return np.array([self.x, self.y, self.z])
    
    def __repr__(self):
        return f"Vec3({self.x:.3f}, {self.y:.3f}, {self.z:.3f})"


class Mat4:
    """4x4 Matrix class for transformations"""
    def __init__(self, data=None):
        if data is None:
            self.m = np.eye(4, dtype=float)
        else:
            self.m = np.array(data, dtype=float).reshape(4, 4)
    
    def __mul__(self, other):
        if isinstance(other, Mat4):
            result = Mat4()
            result.m = self.m @ other.m
            return result
        elif isinstance(other, (list, tuple, np.ndarray)):
            v = np.array(other, dtype=float)
            if len(v) == 3:
                v = np.append(v, 1.0)
            return self.m @ v
        return NotImplemented
    
    @staticmethod
    def identity():
        return Mat4()


def perspective(fov_y, aspect, near, far):
    """Create a perspective projection matrix"""
    f = 1.0 / np.tan(np.radians(fov_y) / 2.0)
    m = Mat4()
    m.m[0, 0] = f / aspect
    m.m[1, 1] = f
    m.m[2, 2] = (far + near) / (near - far)
    m.m[2, 3] = (2.0 * far * near) / (near - far)
    m.m[3, 2] = -1.0
    m.m[3, 3] = 0.0
    return m


def look_at(eye, target, up):
    """Create a look-at view matrix"""
    z = (eye - target).normalize()
    x = up.cross(z).normalize()
    y = z.cross(x)
    
    m = Mat4()
    m.m[0, :3] = [x.x, x.y, x.z]
    m.m[1, :3] = [y.x, y.y, y.z]
    m.m[2, :3] = [z.x, z.y, z.z]
    m.m[0, 3] = -x.dot(eye)
    m.m[1, 3] = -y.dot(eye)
    m.m[2, 3] = -z.dot(eye)
    return m


def viewport(x, y, width, height):
    """Create a viewport transformation matrix"""
    m = Mat4()
    m.m[0, 0] = width / 2.0
    m.m[1, 1] = -height / 2.0
    m.m[0, 3] = x + width / 2.0
    m.m[1, 3] = y + height / 2.0
    m.m[2, 2] = 1.0
    return m

## 6.4 Implementation: Vertex Structure

In [ ]:
class Vertex:
    """Vertex with position, color, normal, and texture coordinates"""
    def __init__(self, position, color=None, normal=None, texcoord=None):
        self.position = Vec3(*position) if not isinstance(position, Vec3) else position
        self.color = np.array(color) if color is not None else np.array([1.0, 1.0, 1.0])
        self.normal = Vec3(*normal).normalize() if normal is not None else Vec3(0, 0, 1)
        self.texcoord = np.array(texcoord) if texcoord is not None else np.array([0.0, 0.0])
    
    def __repr__(self):
        return f"Vertex(pos={self.position}, color={self.color}, normal={self.normal})"

## 6.5 Implementation: 3D Triangle Rasterizer with Perspective Correction

In [ ]:
class Triangle3DRasterizer:
    """Rasterizes 3D triangles with perspective-correct interpolation"""
    
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.framebuffer = np.zeros((height, width, 3), dtype=np.uint8)
        self.zbuffer = np.full((height, width), float('inf'), dtype=float)
    
    def clear(self, color=(0, 0, 0)):
        """Clear framebuffer and z-buffer"""
        self.framebuffer.fill(0)
        self.framebuffer[:] = color
        self.zbuffer.fill(float('inf'))
    
    def set_pixel(self, x, y, depth, color):
        """Set pixel with depth test"""
        if 0 <= x < self.width and 0 <= y < self.height:
            if depth < self.zbuffer[y, x]:
                self.zbuffer[y, x] = depth
                self.framebuffer[y, x] = np.clip(color * 255, 0, 255).astype(np.uint8)
    
    @staticmethod
    def barycentric(p, v0, v1, v2):
        """
        Compute barycentric coordinates of point p with respect to triangle (v0, v1, v2)
        Returns (w0, w1, w2) or None if point is outside triangle
        """
        v0v1 = v1 - v0
        v0v2 = v2 - v0
        v0p = p - v0
        
        d00 = np.dot(v0v1, v0v1)
        d01 = np.dot(v0v1, v0v2)
        d11 = np.dot(v0v2, v0v2)
        d20 = np.dot(v0p, v0v1)
        d21 = np.dot(v0p, v0v2)
        
        denom = d00 * d11 - d01 * d01
        if abs(denom) < 1e-8:
            return None
        
        w1 = (d11 * d20 - d01 * d21) / denom
        w2 = (d00 * d21 - d01 * d20) / denom
        w0 = 1.0 - w1 - w2
        
        # Check if point is inside triangle
        if w0 >= 0 and w1 >= 0 and w2 >= 0:
            return (w0, w1, w2)
        return None
    
    def draw_triangle(self, v0: Vertex, v1: Vertex, v2: Vertex, 
                     screen_coords, depths, smooth_shading=True):
        """
        Draw a 3D triangle with perspective-correct interpolation
        
        Parameters:
        - v0, v1, v2: Vertex objects with attributes
        - screen_coords: [(x0,y0), (x1,y1), (x2,y2)] in screen space
        - depths: [z0, z1, z2] view-space depths
        - smooth_shading: Use Gouraud shading if True, flat shading if False
        """
        # Screen-space coordinates
        p0 = np.array(screen_coords[0])
        p1 = np.array(screen_coords[1])
        p2 = np.array(screen_coords[2])
        
        # View-space depths (for perspective correction)
        z0, z1, z2 = depths[0], depths[1], depths[2]
        
        # Compute bounding box
        min_x = max(0, int(min(p0[0], p1[0], p2[0])))
        max_x = min(self.width - 1, int(max(p0[0], p1[0], p2[0])) + 1)
        min_y = max(0, int(min(p0[1], p1[1], p2[1])))
        max_y = min(self.height - 1, int(max(p0[1], p1[1], p2[1])) + 1)
        
        # Flat shading: compute single color/normal for triangle
        if not smooth_shading:
            flat_color = (v0.color + v1.color + v2.color) / 3.0
        
        # Rasterize
        for y in range(min_y, max_y + 1):
            for x in range(min_x, max_x + 1):
                p = np.array([x + 0.5, y + 0.5])  # Sample at pixel center
                
                # Compute barycentric coordinates
                bary = self.barycentric(p, p0, p1, p2)
                if bary is None:
                    continue
                
                w0, w1, w2 = bary
                
                # Perspective-correct interpolation
                # Interpolate 1/z
                inv_z = w0 / z0 + w1 / z1 + w2 / z2
                z = 1.0 / inv_z
                
                if smooth_shading:
                    # Interpolate color/z
                    color_over_z = (w0 * v0.color / z0 + 
                                   w1 * v1.color / z1 + 
                                   w2 * v2.color / z2)
                    color = color_over_z * z
                else:
                    color = flat_color
                
                # Set pixel with depth test
                self.set_pixel(x, y, z, color)
    
    def show(self, title="3D Rasterization"):
        """Display the framebuffer"""
        plt.figure(figsize=(10, 10))
        plt.imshow(self.framebuffer)
        plt.title(title)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    
    def show_depth(self, title="Depth Buffer"):
        """Visualize the depth buffer"""
        # Normalize depth for visualization
        valid_depths = self.zbuffer[self.zbuffer != float('inf')]
        if len(valid_depths) > 0:
            depth_vis = self.zbuffer.copy()
            depth_vis[depth_vis == float('inf')] = valid_depths.max()
            depth_normalized = (depth_vis - valid_depths.min()) / (valid_depths.max() - valid_depths.min())
        else:
            depth_normalized = np.zeros_like(self.zbuffer)
        
        plt.figure(figsize=(10, 10))
        plt.imshow(depth_normalized, cmap='gray')
        plt.title(title)
        plt.colorbar(label='Normalized Depth')
        plt.axis('off')
        plt.tight_layout()
        plt.show()

## 6.6 Implementation: Simple 3D Pipeline

In [ ]:
class SimplePipeline:
    """Simple 3D rendering pipeline"""
    
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.rasterizer = Triangle3DRasterizer(width, height)
        
        # Default matrices
        self.view_matrix = Mat4.identity()
        self.projection_matrix = Mat4.identity()
        self.viewport_matrix = viewport(0, 0, width, height)
    
    def set_view(self, eye, target, up):
        """Set view matrix using look-at"""
        self.view_matrix = look_at(eye, target, up)
    
    def set_projection(self, fov_y, aspect, near, far):
        """Set perspective projection matrix"""
        self.projection_matrix = perspective(fov_y, aspect, near, far)
    
    def project_vertex(self, vertex: Vertex):
        """
        Transform vertex through the pipeline
        Returns: (screen_coords, view_depth)
        """
        # World to view space
        pos_view = self.view_matrix * [vertex.position.x, vertex.position.y, 
                                       vertex.position.z, 1.0]
        view_depth = -pos_view[2]  # Negative because we look down -Z
        
        # View to clip space
        pos_clip = self.projection_matrix * pos_view
        
        # Perspective divide (clip to NDC)
        if abs(pos_clip[3]) > 1e-8:
            pos_ndc = pos_clip[:3] / pos_clip[3]
        else:
            pos_ndc = pos_clip[:3]
        
        # NDC to screen space
        pos_screen = self.viewport_matrix * [pos_ndc[0], pos_ndc[1], pos_ndc[2], 1.0]
        
        return (pos_screen[0], pos_screen[1]), view_depth
    
    def draw_triangle(self, v0: Vertex, v1: Vertex, v2: Vertex, smooth_shading=True):
        """Draw a 3D triangle through the full pipeline"""
        # Project vertices
        screen0, depth0 = self.project_vertex(v0)
        screen1, depth1 = self.project_vertex(v1)
        screen2, depth2 = self.project_vertex(v2)
        
        # Back-face culling (optional)
        edge1 = np.array(screen1) - np.array(screen0)
        edge2 = np.array(screen2) - np.array(screen0)
        cross = edge1[0] * edge2[1] - edge1[1] * edge2[0]
        if cross <= 0:  # Counter-clockwise = front-facing
            return
        
        # Rasterize
        self.rasterizer.draw_triangle(
            v0, v1, v2,
            [screen0, screen1, screen2],
            [depth0, depth1, depth2],
            smooth_shading=smooth_shading
        )
    
    def clear(self, color=(0, 0, 0)):
        """Clear buffers"""
        self.rasterizer.clear(color)
    
    def show(self, title="Render"):
        """Display result"""
        self.rasterizer.show(title)
    
    def show_depth(self, title="Depth Buffer"):
        """Display depth buffer"""
        self.rasterizer.show_depth(title)

## 6.7 Practical Example 1: Colored Cube with Smooth Shading

In [ ]:
# Create a cube mesh
def create_cube_vertices():
    """Create vertices for a colored cube"""
    vertices = [
        # Front face (red gradient)
        Vertex([-1, -1,  1], color=[1, 0, 0]),
        Vertex([ 1, -1,  1], color=[1, 0.5, 0]),
        Vertex([ 1,  1,  1], color=[1, 1, 0]),
        Vertex([-1,  1,  1], color=[1, 0, 0.5]),
        # Back face (blue gradient)
        Vertex([-1, -1, -1], color=[0, 0, 1]),
        Vertex([ 1, -1, -1], color=[0, 0.5, 1]),
        Vertex([ 1,  1, -1], color=[0, 1, 1]),
        Vertex([-1,  1, -1], color=[0.5, 0, 1]),
    ]
    return vertices

def create_cube_triangles():
    """Define cube triangles as index triplets"""
    return [
        # Front
        (0, 1, 2), (0, 2, 3),
        # Right
        (1, 5, 6), (1, 6, 2),
        # Back
        (5, 4, 7), (5, 7, 6),
        # Left
        (4, 0, 3), (4, 3, 7),
        # Top
        (3, 2, 6), (3, 6, 7),
        # Bottom
        (4, 5, 1), (4, 1, 0),
    ]

# Render the cube
pipeline = SimplePipeline(512, 512)
pipeline.set_view(
    eye=Vec3(3, 3, 5),
    target=Vec3(0, 0, 0),
    up=Vec3(0, 1, 0)
)
pipeline.set_projection(fov_y=60, aspect=1.0, near=0.1, far=100.0)
pipeline.clear(color=(20, 20, 30))

vertices = create_cube_vertices()
triangles = create_cube_triangles()

for tri in triangles:
    pipeline.draw_triangle(vertices[tri[0]], vertices[tri[1]], vertices[tri[2]], smooth_shading=True)

pipeline.show("Colored Cube with Smooth Shading")
pipeline.show_depth("Cube Depth Buffer")

## 6.8 Practical Example 2: Perspective-Correct vs. Linear Interpolation Comparison

In [ ]:
# Create a textured quad at an angle to demonstrate perspective distortion
def create_textured_quad():
    """Create a quad with checkerboard-like color pattern"""
    vertices = [
        Vertex([-2, -1, 0], color=[1, 0, 0]),  # Red
        Vertex([ 2, -1, 0], color=[0, 1, 0]),  # Green
        Vertex([ 2, -1, 8], color=[0, 0, 1]),  # Blue
        Vertex([-2, -1, 8], color=[1, 1, 0]),  # Yellow
    ]
    return vertices, [(0, 1, 2), (0, 2, 3)]

# Render with perspective-correct interpolation
pipeline_correct = SimplePipeline(512, 512)
pipeline_correct.set_view(
    eye=Vec3(0, 3, -2),
    target=Vec3(0, 0, 4),
    up=Vec3(0, 1, 0)
)
pipeline_correct.set_projection(fov_y=60, aspect=1.0, near=0.1, far=100.0)
pipeline_correct.clear(color=(40, 40, 40))

vertices, triangles = create_textured_quad()
for tri in triangles:
    pipeline_correct.draw_triangle(vertices[tri[0]], vertices[tri[1]], vertices[tri[2]], smooth_shading=True)

pipeline_correct.show("Perspective-Correct Interpolation")

print("Notice how colors interpolate correctly across the perspective-distorted quad.")
print("The quad recedes into the distance, and the interpolation accounts for this.")

## 6.9 Practical Example 3: Multiple Overlapping Triangles (Z-Buffer Test)

In [ ]:
# Create multiple overlapping triangles at different depths
def create_overlapping_scene():
    """Create triangles that overlap in screen space but differ in depth"""
    triangles = []
    
    # Triangle 1: Far (blue)
    triangles.append([
        Vertex([-1, -1, 5], color=[0, 0, 1]),
        Vertex([ 1, -1, 5], color=[0, 0.5, 1]),
        Vertex([ 0,  1, 5], color=[0.5, 0.5, 1]),
    ])
    
    # Triangle 2: Middle (green)
    triangles.append([
        Vertex([-0.5, -0.5, 3], color=[0, 1, 0]),
        Vertex([ 1.5, -0.5, 3], color=[0.5, 1, 0]),
        Vertex([ 0.5,  1.5, 3], color=[0.5, 1, 0.5]),
    ])
    
    # Triangle 3: Near (red)
    triangles.append([
        Vertex([ 0, -1, 1], color=[1, 0, 0]),
        Vertex([ 2, -1, 1], color=[1, 0.5, 0]),
        Vertex([ 1,  1, 1], color=[1, 0.5, 0.5]),
    ])
    
    return triangles

# Render overlapping triangles
pipeline_depth = SimplePipeline(512, 512)
pipeline_depth.set_view(
    eye=Vec3(0, 0, -3),
    target=Vec3(0, 0, 0),
    up=Vec3(0, 1, 0)
)
pipeline_depth.set_projection(fov_y=60, aspect=1.0, near=0.1, far=100.0)
pipeline_depth.clear(color=(30, 30, 30))

triangles = create_overlapping_scene()
for tri_verts in triangles:
    pipeline_depth.draw_triangle(tri_verts[0], tri_verts[1], tri_verts[2], smooth_shading=True)

pipeline_depth.show("Overlapping Triangles with Z-Buffer")
pipeline_depth.show_depth("Depth Buffer Visualization")

print("The z-buffer correctly handles depth ordering.")
print("Red triangle (nearest) occludes green, which occludes blue (farthest).")

## 6.10 Practical Example 4: Rotating Pyramid Animation

In [ ]:
def rotation_matrix_y(angle_deg):
    """Create rotation matrix around Y axis"""
    angle = np.radians(angle_deg)
    c, s = np.cos(angle), np.sin(angle)
    m = Mat4()
    m.m[0, 0] = c
    m.m[0, 2] = s
    m.m[2, 0] = -s
    m.m[2, 2] = c
    return m

def create_pyramid():
    """Create a pyramid mesh"""
    vertices = [
        # Base vertices
        Vertex([-1, 0, -1], color=[1, 0, 0]),
        Vertex([ 1, 0, -1], color=[0, 1, 0]),
        Vertex([ 1, 0,  1], color=[0, 0, 1]),
        Vertex([-1, 0,  1], color=[1, 1, 0]),
        # Apex
        Vertex([ 0, 2,  0], color=[1, 1, 1]),
    ]
    
    triangles = [
        # Base
        (0, 1, 2), (0, 2, 3),
        # Sides
        (0, 4, 1),
        (1, 4, 2),
        (2, 4, 3),
        (3, 4, 0),
    ]
    
    return vertices, triangles

# Render pyramid at different rotation angles
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
angles = [0, 45, 90, 135, 180, 270]

for idx, angle in enumerate(angles):
    ax = axes[idx // 3, idx % 3]
    
    # Create pipeline
    pipeline_anim = SimplePipeline(256, 256)
    pipeline_anim.set_view(
        eye=Vec3(3, 3, 5),
        target=Vec3(0, 1, 0),
        up=Vec3(0, 1, 0)
    )
    pipeline_anim.set_projection(fov_y=60, aspect=1.0, near=0.1, far=100.0)
    pipeline_anim.clear(color=(25, 25, 35))
    
    # Create and rotate pyramid
    base_vertices, triangles = create_pyramid()
    rot_matrix = rotation_matrix_y(angle)
    
    # Transform vertices
    rotated_vertices = []
    for v in base_vertices:
        pos = rot_matrix * [v.position.x, v.position.y, v.position.z, 1.0]
        rotated_vertices.append(Vertex(pos[:3], color=v.color))
    
    # Draw triangles
    for tri in triangles:
        pipeline_anim.draw_triangle(
            rotated_vertices[tri[0]], 
            rotated_vertices[tri[1]], 
            rotated_vertices[tri[2]], 
            smooth_shading=True
        )
    
    # Display
    ax.imshow(pipeline_anim.rasterizer.framebuffer)
    ax.set_title(f"Rotation: {angle}°")
    ax.axis('off')

plt.tight_layout()
plt.suptitle("Rotating Pyramid with Perspective-Correct Shading", y=1.02, fontsize=16)
plt.show()

print("Notice how the color gradients remain consistent as the pyramid rotates.")
print("This demonstrates perspective-correct attribute interpolation.")

## Summary

In this chapter, we implemented:

1. **Perspective-Correct Interpolation**: Proper handling of attribute interpolation in perspective projection by interpolating $a/z$ and $1/z$ separately

2. **Depth Interpolation**: Correct z-value computation using barycentric coordinates for depth testing

3. **3D Triangle Rasterizer**: Complete implementation that handles:
   - Barycentric coordinate computation
   - Perspective-correct color interpolation
   - Z-buffer depth testing
   - Both smooth and flat shading

4. **3D Rendering Pipeline**: Integration of view, projection, and viewport transformations

5. **Practical Examples**:
   - Colored cube with smooth shading
   - Perspective correction demonstration
   - Overlapping triangles with depth testing
   - Animated rotating pyramid

These techniques form the foundation of modern rasterization-based rendering. In the next chapter, we'll add **shading and illumination** models to create more realistic images.